<div style="text-align: center">
  <img src="https://github.com/KarolisRam/MineRL2021-Intro-baselines/blob/main/img/colab_banner.png?raw=true">
</div>

# Introduction
This notebook contains the Behavioural Cloning baselines for the Research track of the [MineRL 2021](https://minerl.io/) competition. To run it you will need to enable GPU by going to `Runtime -> Change runtime type` and selecting GPU from the drop down list.

These baselines differ slightly from the standalone version of these baselines on github - the DATA_SAMPLES parameter is set to 400,000 instead of the default 1,000,000. This is done to fit into the RAM limits of Colab.

To train the agent using the obfuscated action space we first discretize the action space using KMeans clustering. We then train the agent using Behavioural cloning. The training takes 10-15 mins.

You can find more details about the obfuscation here:  
[K-means exploration](https://minerl.io/docs/tutorials/k-means.html)

Also see the in-depth analysis of the obfuscation and the KMeans approach done by one of the teams in the 2020 competition:

[Obfuscation and KMeans analysis](https://github.com/GJuceviciute/MineRL-2020)

Please note that any attempt to work with the obfuscated state and action spaces should be general and work with a different dataset or even a completely new environment.

# Setup

In [1]:
skip_install = True

In [2]:

if not skip_install:
    #%%capture
    !add-apt-repository -y ppa:openjdk-r/ppa
    !apt-get -y purge openjdk-*
    !apt-get -y install openjdk-8-jdk
    !apt-get -y install xvfb xserver-xephyr vnc4server python-opengl ffmpeg

In [3]:
if not skip_install:
    !pip3 install --upgrade minerl
    !pip3 install pyvirtualdisplay
    !pip3 install torch
    !pip3 install scikit-learn
    !pip3 install -U colabgymrender

# Import Libraries

In [4]:
import random
import numpy as np
import torch as th
from torch import nn
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
#from pyvirtualdisplay import Display
from sklearn.cluster import KMeans
import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.
import pickle

/home/olli/gits/ISY_MINE/bc_baseline/env/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [26]:
from os.path import join, isfile

from os import makedirs
# Parameters:
EPOCHS = 4  # how many times we train over dataset.
LEARNING_RATE = 0.0001  # Learning rate for the neural network.

# i didnt change this. seems to work fine.
BATCH_SIZE = 32

NUM_ACTION_CENTROIDS = 70  # Number of KMeans centroids used to cluster the data.

DATA_SAMPLES = 400000  # how many samples to use from the dataset. Impacts RAM usage

# you can save checkpoints after x epochs. Put them in a list.
# They will be saved in the model folder
checkpoints=[]



# Bypass filter and use whole dataset
BYPASS_FILTER = False

# Filter frames. Use window-before and window-after frames before and after
# current frame to count present rewards
WINDOW_BEFORE=100
WINDOW_AFTER=20

# ignore window and take all frames between rewards
USE_ALL = True 

# Latent dimension of pov. Use 1,4 or 8
LATENT_PIC_DIMENSION=4

## Use small Dataset for debugging
DEBUG=False

## EVAL is not implemented correctly
EVAL=False

# Dataset to use. 
DATASET = "MineRLObtainIronPickaxeVectorObf-v0"

NAME = "cobblestone"

# Datadir
DATADIR = "data"

# pkl filepath
DATASET_FILE=join(DATADIR,f'{NAME}.pkl')

# number of replays to use
# 100 works with 16GB ram... 
NO_REPLAYS = 100

# Overwrite old pkl always
ALWAYS_BUILD_DATASET = False

if th.cuda.is_available():
    dev = th.device('cuda')
else:
    dev = th.device('cpu')

# define paths etc.
MODEL_NAME = f'{NAME}_window-before={WINDOW_BEFORE}_window-after={WINDOW_AFTER}_latent-pic-dimension={LATENT_PIC_DIMENSION}_epochs={EPOCHS}_clusters={NUM_ACTION_CENTROIDS}'
try:
    makedirs(MODEL_NAME,exist_ok=False)
except:
    print("WARINING! MODEL ALREADY PRESENT. OLD MODEL WILL BE OVERWRITTEN!")
TRAIN_MODEL_NAME=join(MODEL_NAME,'research_potato.pth')
TRAIN_KMEANS_MODEL_NAME= join(MODEL_NAME,'centroids_for_research_potato.npy')
TEST_MODEL_NAME = TRAIN_MODEL_NAME
TEST_KMEANS_MODEL_NAME = TRAIN_KMEANS_MODEL_NAME


# Neural network

In [19]:
class NatureCNN(nn.Module):
    """
    
    I changed the net slightly to make it use 1x1, 4x4 and 8x8 latend picture dimension.
    I also doubled channel count.
    
    CNN from DQN nature paper:
        Mnih, Volodymyr, et al.
        "Human-level control through deep reinforcement learning."
        Nature 518.7540 (2015): 529-533.

    Nicked from stable-baselines3:
        https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/torch_layers.py

    :param input_shape: A three-item tuple telling image dimensions in (C, H, W)
    :param output_dim: Dimensionality of the output vector
    :param latent_pic_dim: choose one of 3 models with values 1,4,8
    """

    def __init__(self, input_shape, output_dim,latent_pic_dim=4):
        super().__init__()
        n_input_channels = input_shape[0]
        
        if latent_pic_dim ==8:
            
            self.cnn = nn.Sequential(
                nn.Conv2d(n_input_channels, 64, kernel_size=8, stride=4, padding=0),
                nn.ReLU(),
                nn.Conv2d(64, 64, kernel_size=4, stride=1, padding=0),
                nn.ReLU(),
                nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
                nn.ReLU(),
                nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0),
                nn.ReLU(),
                nn.Flatten()

            )

        elif latent_pic_dim == 4:
            self.cnn = nn.Sequential(
                nn.Conv2d(n_input_channels, 64, kernel_size=8, stride=4, padding=0),
                nn.ReLU(),
                nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=0),
                nn.ReLU(),
                nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
                nn.ReLU(),
                nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0),
                nn.ReLU(),
                nn.Flatten()

            )
        elif latent_pic_dim ==1:
            self.cnn = nn.Sequential(
                nn.Conv2d(n_input_channels, 64, kernel_size=8, stride=4, padding=0),
                nn.ReLU(),
                nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=0),
                nn.ReLU(),
                nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0),
                nn.ReLU(),
                nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0),
                nn.ReLU(),
                nn.Flatten()

            )
        else:
            print("You can only use 8,4, or 1 as latent pic dim!")
            exit(-1)
            
            
            
        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.zeros(1, *input_shape)).shape[1]

        self.linear = nn.Sequential(
            nn.Linear(n_flatten, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, output_dim)
        )

    def forward(self, observations: th.Tensor) -> th.Tensor:
        
        return self.linear(self.cnn(observations))

#test code
#net = NatureCNN((3, 64, 64),100,8)    


# Setup training

In [20]:
def filter_actions_ix_new(trajectory_ix ,rewards,max_reward=16,reward_before=8,before_count=40,after_count=40,bypass=False):
    """
    Filter out frames that dont have ${min_rewards} in the window vicinity.
    Window = [beforecount:i:afterafter_count]
    
    param actions: list of all actions to filter
    param trajectory_ix: the start index of each episode. used to account for overlapping windows
    param rewards: list of rewards, matching actions
    param min_reward: min reward that should be present in the window
    param before_count: window to the left(past)
    param before_count: window to the right(future)
    param bypass: set to True to not filter

    """
    # add 0 as starting point.
    trajectory_ix.insert(0,0)

    reward_before_ix = -1
    max_reward_ix = -1
    reward_windows = []

    
    if bypass :
        return list(range(len(actions)))
    filtered_ix = []
    for i,current_trajectory in tqdm(enumerate(trajectory_ix),total=len(trajectory_ix)-1):
        if i+1>= len(trajectory_ix):
            next_trajectory = len(rewards)
        else:    
            next_trajectory = trajectory_ix[i+1]
        
        second_reward = False    
        for j,rew in (enumerate(rewards[current_trajectory:next_trajectory])):


            if max_reward == reward_before:
                if rewards[current_trajectory+j] == max_reward and second_reward: 
                    max_reward_ix= current_trajectory+j
                if rewards[current_trajectory+j] == reward_before and not second_reward:
                    reward_before_ix = current_trajectory+j
                    second_reward = True
            else:
                if rewards[current_trajectory+j] == max_reward: 
                    max_reward_ix= current_trajectory+j
                if rewards[current_trajectory+j] == reward_before:
                    reward_before_ix = current_trajectory+j
                
       
        
        reward_windows.append(max_reward_ix-reward_before_ix)
        
        if max_reward_ix + after_count > next_trajectory:
            after_ix = next_trajectory
        else:
            after_ix = max_reward_ix + after_count
            
        if max_reward_ix - before_count < current_trajectory:   
            before_ix = current_trajectory
        else:
            before_ix = max_reward_ix - before_count
        
        if USE_ALL :
            before_ix = reward_before_ix
            after_ix = max_reward_ix
            
        
        filtered_ix.extend(list(range(before_ix,after_ix)))
    #print(len(reward_windows),np.mean(reward_windows),np.std(reward_windows))  
    return filtered_ix



            

In [21]:
def load_and_pickle_dataset():
    
    if not isfile(DATASET_FILE) or ALWAYS_BUILD_DATASET:
        minerl.data.download(directory=DATADIR, environment=DATASET);
        data = minerl.data.make(DATASET,  data_dir=DATADIR, num_workers=1)
        trajectory_names = data.get_trajectory_names()
        #random.shuffle(trajectory_names)

        trajectory_lens = []
        trajectories = []
        
        
        for trajectory_name in trajectory_names[0:NO_REPLAYS]:
            trajectory = data.load_data(trajectory_name, skip_interval=0, include_metadata=False)
            del data
            data = minerl.data.make(DATASET,  data_dir=DATADIR, num_workers=1)
            trajectory = list(trajectory)
            
            
            rewards = []
            for _, _, dataset_reward, _, _ in trajectory:
                rewards.append(dataset_reward)
            
            filtered_ix = filter_actions_ix_new([len(trajectory)],rewards,max_reward=2,reward_before=1,before_count=WINDOW_BEFORE,after_count=WINDOW_AFTER)
           
            trajectory=np.array(trajectory)
            trajectory = trajectory[filtered_ix]
            trajectory=list(trajectory)
           
            
            
            trajectory_lens.append(len((trajectory)))
            trajectories.append(trajectory)
            del trajectory

        f = open(DATASET_FILE,'wb')
        pickle.dump(trajectories,f,protocol=pickle.HIGHEST_PROTOCOL)
        f.close()
    else:
        print('Using old dataset :)')
   
def load_from_pkl():
    f = open(DATASET_FILE,'rb')

    trajectories = pickle.load(f)
    return trajectories

In [22]:
def filter_actions_ix(trajectory_ix ,rewards,min_reward=2,before_count=40,after_count=40,bypass=False):
    """
    Filter out frames that dont have ${min_rewards} in the window vicinity.
    Window = [beforecount:i:afterafter_count]
    
    param actions: list of all actions to filter
    param trajectory_ix: the start index of each episode. used to account for overlapping windows
    param rewards: list of rewards, matching actions
    param min_reward: min reward that should be present in the window
    param before_count: window to the left(past)
    param before_count: window to the right(future)
    param bypass: set to True to not filter

    """
    # add 0 as starting point.
    trajectory_ix.insert(0,0)
    print(bypass)
    if bypass :
        return list(range(len(actions)))
    filtered_ix = []
    for i,current_trajectory in tqdm(enumerate(trajectory_ix),total=len(trajectory_ix)-1):
        if i+1>= len(trajectory_ix):
            next_trajectory = len(actions)
        else:    
            next_trajectory = trajectory_ix[i+1]
        for i,act in (enumerate(actions[current_trajectory:next_trajectory])):

          if i-before_count<0:
            before_ix = 0
          else:
            before_ix = i-before_count
          if i+after_count>len(actions):
            after_ix = len(actions)-1
          else:
            after_ix = i+after_count
          if sum(rewards[before_ix:after_ix]) > min_reward:
            filtered_ix.append(current_trajectory+i)

    print(f'Using {len(filtered_ix)} of {len(actions)} samples!')    
    assert len(np.unique(filtered_ix) == len(filtered_ix)), 'WARNING: Duplicate samples!'
    return filtered_ix

def cluster(actions):
    """
    
    run sklearn.KMEANS() on actions. Uses NUM_ACTION_CENTROIDS global.
    
    """
    
    print("Running KMeans on the action vectors")
    kmeans = KMeans(n_clusters=NUM_ACTION_CENTROIDS,verbose=1)
    kmeans.fit(actions)
    action_centroids = kmeans.cluster_centers_
    return action_centroids
    print("KMeans done")

In [23]:
def train():
    


    # First, use k-means to find actions that represent most of them.
    # This proved to be a strong approach in the MineRL 2020 competition.
    # See the following for more analysis:
    # https://github.com/GJuceviciute/MineRL-2020

    # Go over the dataset once and collect all actions and the observations (the "pov" image).
    # We do this to later on have uniform sampling of the dataset and to avoid high memory use spikes.
    all_actions = []
    all_pov_obs = []
    all_rewards = []
    

    # load saved dataset from pkl
    trajectories = load_from_pkl()
    
    
    # Add trajectories to the data until we reach the required DATA_SAMPLES
    

    
    for trajectory in trajectories:        
        for dataset_observation, dataset_action, dataset_reward, _, _ in trajectory:
            all_actions.append(dataset_action["vector"])
            all_pov_obs.append(dataset_observation["pov"])
            all_rewards.append(dataset_reward)
        if len(all_actions) >= DATA_SAMPLES:
            break
        del trajectory     
    del trajectories    

    all_actions = np.array(all_actions)
    all_pov_obs = np.array(all_pov_obs)
    
    print(f'Training on {len(all_actions)} samples')
    
    
    # apply filtering of 'low reward' actions

    #trajectory_ix = np.cumsum(trajectory_lens)
    #ix = filter_actions_ix_new(all_actions,list(trajectory_ix),all_rewards,bypass=BYPASS_FILTER)


    filtered_actions = all_actions
    filtered_pov_obs = all_pov_obs



    # Run k-means clustering using scikit-learn.  
    action_centroids = cluster(filtered_actions)


    # Now onto behavioural cloning itself.
    # Much like with intro track, we do behavioural cloning on the discrete actions,
    # where we turn the original vectors into discrete choices by mapping them to the closest
    # centroid (based on Euclidian distance).

    network = NatureCNN((3, 64, 64), NUM_ACTION_CENTROIDS,LATENT_PIC_DIMENSION).to(dev)
    optimizer = th.optim.Adam(network.parameters(), lr=LEARNING_RATE)
    loss_function = nn.CrossEntropyLoss()

    num_samples = filtered_actions.shape[0]
    update_count = 0
    losses = []
    # We have the data loaded up already in all_actions and all_pov_obs arrays.
    # Let's do a manual training loop
    print("Training")
    for e in range(EPOCHS):
        print(f"starting epoch {e+1}")
        # Randomize the order in which we go over the samples
        epoch_indices = np.arange(num_samples)
        np.random.shuffle(epoch_indices)
        for batch_i in range(0, num_samples, BATCH_SIZE):
            # NOTE: this will cut off incomplete batches from end of the random indices
            batch_indices = epoch_indices[batch_i:batch_i + BATCH_SIZE]

            # Load the inputs and preprocess
            obs = filtered_pov_obs[batch_indices].astype(np.float32)
            # Transpose observations to be channel-first (BCHW instead of BHWC)
            obs = obs.transpose(0, 3, 1, 2)
            # Normalize observations. Do this here to avoid using too much memory (images are uint8 by default)
            obs /= 255.0

            # Map actions to their closest centroids
            action_vectors = filtered_actions[batch_indices]
            # Use numpy broadcasting to compute the distance between all
            # actions and centroids at once.
            # "None" in indexing adds a new dimension that allows the broadcasting
            distances = np.sum((action_vectors - action_centroids[:, None]) ** 2, axis=2)
            # Get the index of the closest centroid to each action.
            # This is an array of (batch_size,)
            actions = np.argmin(distances, axis=0)

            # Obtain logits of each action
            logits = network(th.from_numpy(obs).float().to(dev))

            # Minimize cross-entropy with target labels.
            # We could also compute the probability of demonstration actions and
            # maximize them.
            loss = loss_function(logits, th.from_numpy(actions).long().to(dev))

            # Standard PyTorch update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            update_count += 1
            losses.append(loss.item())
            if (update_count % 100) == 0:
                mean_loss = sum(losses) / len(losses)
                tqdm.write("Iteration {}. Loss {:<10.3f}".format(update_count, mean_loss))
                losses.clear()
        
        # save checkpoints.
        if (e+1) in checkpoints:
            print(f'saving_checkpoint_epoch{e+1}')
            th.save(network.state_dict(), f"{TRAIN_MODEL_NAME}_checkpoint_epoch={e+1}")
            
    print("Training done")

    # Save network and the centroids into separate files
    np.save(TRAIN_KMEANS_MODEL_NAME, action_centroids)
    th.save(network.state_dict(), TRAIN_MODEL_NAME)


   

# Download the data

In [24]:
#%%capture
load_and_pickle_dataset()

100%|███████████████████████████████████████████████████████████████████████████████████████| 5521/5521 [00:00<00:00, 147025.13it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████| 12873/12873 [00:00<00:00, 59882.01it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████| 3524/3524 [00:00<00:00, 146912.58it/s]


...

  0%|          | 0/1 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████| 6825/6825 [00:00<00:00, 147813.35it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████| 7374/7374 [00:00<00:00, 88722.38it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

# Train

In [27]:
# inertia should be ~60-70 for 100 centroids, ~90-100 for 70, ~200 for 40. Depends on random state i think
# filtered dataset get lower inertia :) so it seems to be good
train()  # only need to run this once.


Training on 121946 samples
Running KMeans on the action vectors
Initialization complete
Iteration 0, inertia 30.70589600168103
Iteration 1, inertia 27.650632820099414
Iteration 2, inertia 27.179617220905634
Iteration 3, inertia 26.94624607919755
Iteration 4, inertia 26.873829923187213
Iteration 5, inertia 26.84748257749483
Iteration 6, inertia 26.836363010787064
Iteration 7, inertia 26.83108337671643
Iteration 8, inertia 26.824938769980147
Iteration 9, inertia 26.81134516258769
Iteration 10, inertia 26.800226867268577
Iteration 11, inertia 26.782088657391085
Iteration 12, inertia 26.766209541674947
Iteration 13, inertia 26.73557800493458
Iteration 14, inertia 26.674411184473797
Iteration 15, inertia 26.62857476859245
Iteration 16, inertia 26.615273728727207
Iteration 17, inertia 26.611800207392285
Iteration 18, inertia 26.60998260695445
Iteration 19, inertia 26.608503414421275
Iteration 20, inertia 26.607723903564573
Iteration 21, inertia 26.60650574366849
Iteration 22, inertia 26.6058

Iteration 22, inertia 26.91373048327273
Converged at iteration 22: center shift 4.660240468487226e-08 within tolerance 6.549573422333151e-08.
Initialization complete
Iteration 0, inertia 31.835424865252584
Iteration 1, inertia 28.437068232637678
Iteration 2, inertia 27.943359780935282
Iteration 3, inertia 27.722930856179794
Iteration 4, inertia 27.674426707842763
Iteration 5, inertia 27.656695204485032
Iteration 6, inertia 27.64985762298413
Iteration 7, inertia 27.643830908448738
Iteration 8, inertia 27.640500062370517
Iteration 9, inertia 27.636725842360722
Iteration 10, inertia 27.631073285156123
Iteration 11, inertia 27.628722593188492
Iteration 12, inertia 27.627996591175783
Iteration 13, inertia 27.626258330848323
Iteration 14, inertia 27.624747110529242
Iteration 15, inertia 27.62287848084106
Iteration 16, inertia 27.621048628786692
Iteration 17, inertia 27.61902112547532
Iteration 18, inertia 27.618069760100312
Iteration 19, inertia 27.617881703922166
Converged at iteration 19: 